In [1]:
from sklearn import svm
import pickle
import numpy as np
import os
from sklearn.utils import multiclass as mc
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, scale
from lib import accuracy
from lib import labels

In [2]:
data_dir = "../data/"
pickle_file = 'tf_data_max.pickle'

with open(data_dir + pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = np.matrix(save['train_data'], dtype='float32')
    train_labels = np.matrix(save['train_labels'], dtype='float32')
    valid_dataset = np.matrix(save['validate_data'], dtype='float32')
    valid_labels = np.matrix(save['validate_labels'], dtype='float32')
    test_dataset = np.matrix(save['test_data'], dtype='float32')
    test_bids = list(save['test_business_ids'])
    #test_bids = np.ravel(test_bids)
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape
    print 'Test set', test_dataset.shape, len(test_bids)

# Convert labels to a dict of binarized labels [1. if true, 1. if false]
# So can be used for softmax per label.
train_labels = labels.binarize_softmax_labels(train_labels)
valid_labels = labels.binarize_softmax_labels(valid_labels)

Training set (1600, 1024) (1600, 9)
Validation set (400, 1024) (400, 9)
Test set (10000, 1024) 10000


In [3]:
# Disabled scaling because it seems to make it always a little worse!
scaled_train = train_dataset #scale(train_dataset)
scaled_valid = valid_dataset #scale(valid_dataset)
svm_preds = np.zeros((400, 9), dtype="float32")
svm_test_preds = np.zeros((test_dataset.shape[0], 9), dtype="float32")

for i in range(9):
    label_n = i
    trainL = labels.matrix_2_labels(train_labels[label_n])

    trainL = MultiLabelBinarizer().fit_transform(trainL)
    #train_labels = matrix_2_labels(train_labels)
    trainL = np.asarray(trainL)

    #clf = svm.SVC(decision_function_shape='ovr', kernel='linear')
    

    clf = OneVsRestClassifier(svm.LinearSVC(random_state=42)).fit(scaled_train, trainL)
    predictions = clf.predict(scaled_valid)
    test_predictions = clf.predict(test_dataset)
    print "#"+str(i)+": {0:.2f}".format(accuracy.binarized_accuracy(predictions, valid_labels[label_n]) * 100)
    svm_preds[:,i] = predictions[:,0]
    svm_test_preds[:,i] = test_predictions[:,0]

with open(data_dir+"svm_test_preds.pickle", 'wb') as f:
    # Pickle dictionary using protocol 0.
    pickle.dump(svm_test_preds, f)
#0: good_for_lunch
#1: good_for_dinner
#2: takes_reservations
#3: outdoor_seating
#4: restaurant_is_expensive
#5: has_alcohol
#6: has_table_service
#7: ambience_is_classy
#8: good_for_kids    
    
    
# svm.LinearSVC(random_state=0)
#SVM by label (mean)
#0: 72.25
#1: 79.50
#2: 82.75
#3: 62.50
#4: 84.25
#5: 83.75
#6: 88.5
#7: 83.25
#8: 83.75

# svm.LinearSVC(random_state=0)
#SVM by label (max)
#0: 66.25
#1: 78.75
#2: 83.75
#3: 59.50
#4: 83.50
#5: 84.25
#6: 87.50
#7: 78.25
#8: 78.75

# Best DNN Model (current)
# 2 Hidden Layer model with L2 regularization added to the loss, and 20% dropout.
# AdaGradOptimizer
# regularizer = 1e-8
# training_rate = 0.5
# num_steps = 3000 (more steps could have more improvement!)
#0: 77.75%
#1: 84.50
#2: 86.50
#3: 59.25
#4: 86.50
#5: 87.25
#6: 92.00
#7: 84.00
#8: 86.00


#0: 73.00
#1: 80.50
#2: 84.00
#3: 62.25
#4: 84.50
#5: 83.50
#6: 88.25
#7: 83.25
#8: 84.00


In [4]:
svm_test_preds.shape

(10000, 9)